In [1]:
import os
import sys
import pickle
import numpy as np
import torch 
from torch import nn
from tqdm import tqdm

os.environ["CONFIG_PATHS"] = "../configs/small_model.yaml"
os.environ["CONFIG_OVERRIDES"] = 'game.moves_directory="../data/moves_10"'
sys.path.append("../src")

from configuration import config
from neural_net import NeuralNet

""

Loaded config:  {"development": {"debug_mode": true, "profile": false, "runtime": 32400, "display_logs_in_console": false, "output_directory": "data/", "log_mcts_report_fraction": 0.01}, "game": {"board_size": 10, "num_moves": 6233, "moves_directory": "../data/moves_10"}, "architecture": {"gameplay_processes": 6, "coroutines_per_process": 256}, "networks": {"default": {"main_body_channels": 64, "value_head_channels": 16, "value_head_flat_layer_width": 64, "policy_head_channels": 64, "residual_blocks": 8, "model_path": "", "batch_size": 128}}, "agents": [{"mcts": {"network": "default", "full_move_probability": 0.2, "full_move_rollouts": 500, "fast_move_rollouts": 100, "ucb_exploration": 1.4, "total_dirichlet_alpha": 10.83, "root_exploration_fraction": 0.25}, "name": "default"}, {"mcts": {"network": "default", "full_move_probability": 0.2, "full_move_rollouts": 500, "fast_move_rollouts": 100, "ucb_exploration": 1.4, "total_dirichlet_alpha": 10.83, "root_exploration_fraction": 0.25}, "nam

''

In [2]:
BOARD_SIZE = config()["game"]["board_size"]

In [3]:
def model_size(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum(p.numel() for p in model_parameters)

In [4]:
import time

def time_per_eval(num_evaluations, batch_size, model):
    num_batches = num_evaluations // batch_size
    num_evaluations = num_batches * batch_size

    random_arrays = np.random.random((num_batches, batch_size, 4, BOARD_SIZE, BOARD_SIZE))
    print("Starting...")

    start = time.perf_counter()
    for i in range(num_batches):
        occupancies = torch.from_numpy(random_arrays[i]).to(device="mps", dtype=torch.float32)
        model(occupancies)
    elapsed = time.perf_counter() - start

    return elapsed / (num_batches * batch_size)

In [6]:
model = NeuralNet(config()["networks"]["default"], load_model=False)
print("Model size:", model_size(model))

Model size: 40600909


In [11]:
time_per_eval(
    num_evaluations=10000,
    batch_size=128,
    model=model.to("mps"),
)

Starting...


6.861165785173794e-05

In [7]:
torch.save(model.state_dict(), "../data/misc/untrained_model.pt")